In [67]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

In [68]:
bike = pd.read_csv(r"C:\Users\Kavita manoj\Downloads\BikeSharing - Day.csv")

In [69]:
bike = bike.drop(columns=['instant', 'dteday'])
bike.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [70]:
bike.mnth = bike.mnth.astype(str)
bike.weekday = bike.weekday.astype(str)
bike.season = bike.season.astype(str)
bike.weathersit = bike.weathersit.astype(str)

In [71]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
bikeQuant = add_constant(pd.DataFrame({'temp': bike.temp, 'atemp': bike.atemp, 'hum': bike.hum, 'windspeed': bike.windspeed}))

In [72]:
vif = pd.DataFrame()
vif['feature_vars'] = bikeQuant.columns
vif['VIF'] = [variance_inflation_factor(bikeQuant, i) for i in range(len(bikeQuant.columns))]

In [73]:
vif

,feature_vars,VIF
0,const,45.633543
1,temp,62.969819
2,atemp,63.632351
3,hum,1.079267
4,windspeed,1.126768


In [74]:
bikeQuant = bikeQuant.drop(columns='atemp')
vif2 = pd.DataFrame()
vif2['feature_vars'] = bikeQuant.columns
vif2['VIF'] = [variance_inflation_factor(bikeQuant, i) for i in range(len(bikeQuant.columns))]

In [75]:
vif2

,feature_vars,VIF
0,const,41.075299
1,temp,1.034283
2,hum,1.074850
3,windspeed,1.084580


In [76]:
from sklearn.preprocessing import scale
for i in bikeQuant.drop(columns='const').columns:
    bike[i+'_s'] = scale(bikeQuant[i], with_mean=True, with_std=True)

In [77]:
bike.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,temp_s,hum_s,windspeed_s
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985,-0.826662,1.250171,-0.387892
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801,-0.721095,0.479113,0.749602
2,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349,-1.634657,-1.339274,0.746632
3,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562,-1.614780,-0.263182,-0.389829
4,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600,-1.467414,-1.341494,-0.046307


In [78]:
ohe = pd.get_dummies(data=bike, columns=['season', 'mnth', 'weekday', 'weathersit'])

In [79]:
ohe.head()

,yr,holiday,workingday,temp,atemp,hum,windspeed,casual,registered,cnt,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weathersit_1,weathersit_2,weathersit_3
0,0,0,0,0.344167,0.363625,0.805833,0.160446,331,654,985,...,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0.363478,0.353739,0.696087,0.248539,131,670,801,...,1,0,0,0,0,0,0,0,1,0
2,0,0,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349,...,0,1,0,0,0,0,0,1,0,0
3,0,0,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562,...,0,0,1,0,0,0,0,1,0,0
4,0,0,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600,...,0,0,0,1,0,0,0,1,0,0


In [81]:
ohe.columns

Index(['yr', 'holiday', 'workingday', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt', 'temp_s', 'hum_s', 'windspeed_s',
       'season_1', 'season_2', 'season_3', 'season_4', 'mnth_1', 'mnth_10',
       'mnth_11', 'mnth_12', 'mnth_2', 'mnth_3', 'mnth_4', 'mnth_5', 'mnth_6',
       'mnth_7', 'mnth_8', 'mnth_9', 'weekday_0', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weathersit_1',
       'weathersit_2', 'weathersit_3'],
      dtype='object')

In [83]:
ohe = ohe.drop(columns=['casual', 'registered', 'yr'])
formula = ''
for i in ohe.columns:
    formula+= i+'+'
print(formula)

holiday+workingday+temp+atemp+hum+windspeed+cnt+temp_s+hum_s+windspeed_s+season_1+season_2+season_3+season_4+mnth_1+mnth_10+mnth_11+mnth_12+mnth_2+mnth_3+mnth_4+mnth_5+mnth_6+mnth_7+mnth_8+mnth_9+weekday_0+weekday_1+weekday_2+weekday_3+weekday_4+weekday_5+weekday_6+weathersit_1+weathersit_2+weathersit_3+


In [84]:
smf.ols('cnt~holiday+workingday+temp_s+hum_s+windspeed_s+season_1+season_2+season_3+season_4+mnth_1+mnth_10+mnth_11+mnth_12+mnth_2+mnth_3+mnth_4+mnth_5+mnth_6+mnth_7+mnth_8+mnth_9+weekday_0+weekday_1+weekday_2+weekday_3+weekday_4+weekday_5+weekday_6+weathersit_1+weathersit_2+weathersit_3', data=ohe).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.587
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     38.54
Date:                Mon, 24 Apr 2023   Prob (F-statistic):          8.03e-117
Time:                        14:13:08   Log-Likelihood:                -6246.2
No. Observations:                 731   AIC:                         1.255e+04
Df Residuals:                     704   BIC:                         1.267e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     9.146e+15   8.24e+15      1.109      0.268   -7.04e+15    2.53e+16
holiday      -4.109e+15    3.7e+15     -1.109      0.268   -1.14e+16    3.16e+15
workingday   -4.109e+15    3.7e+15     -1.109      0.268   -1.14e+16    3.16e+15
temp_s        1234.5520    122.663     10.065      0.000     993.723    1475.381
hum_s         -435.2247     67.836     -6.416      0.000    -568.410    -302.039
windspeed_s   -276.0990     51.752     -5.335      0.000    -377.705    -174.493
season_1      2.218e+15      2e+15      1.109      0.268   -1.71e+15    6.14e+15
season_2      2.218e+15      2e+15      1.109      0.268   -1.71e+15    6.14e+15
season_3      2.218e+15      2e+15      1.109      0.268   -1.71e+15    6.14e+15
season_4      2.218e+15      2e+15      1.109      0.268   -1.71e+15    6.14e+15
mnth_1        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_10       6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_11       6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_12       6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_2        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_3        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_4        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_5        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_6        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_7        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_8        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
mnth_9        6.653e+14      6e+14      1.109      0.268   -5.12e+14    1.84e+15
weekday_0    -1.515e+15   1.37e+15     -1.109      0.268    -4.2e+15    1.17e+15
weekday_1     2.593e+15   2.34e+15      1.109      0.268      -2e+15    7.18e+15
weekday_2     2.593e+15   2.34e+15      1.109      0.268      -2e+15    7.18e+15
weekday_3     2.593e+15   2.34e+15      1.109      0.268      -2e+15    7.18e+15
weekday_4     2.593e+15   2.34e+15      1.109      0.268      -2e+15    7.18e+15
weekday_5     2.593e+15   2.34e+15      1.109      0.268      -2e+15    7.18e+15
weekday_6    -1.515e+15   1.37e+15     -1.109      0.268    -4.2e+15    1.17e+15
weathersit_1 -1.051e+16   9.48e+15     -1.109      0.268   -2.91e+16    8.09e+15
weathersit_2 -1.051e+16   9.48e+15     -1.109      0.268   -2.91e+16    8.09e+15
weathersit_3 -1.051e+16   9.48e+15     -1.109      0.268   -2.91e+16    8.09e+15
==============================================================================
Omnibus:                       45.657   Durbin-Watson:                   0.453
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.251
Skew:      